In [13]:
import matplotlib.pyplot as plt
import numpy as np
import pyresample
from netCDF4 import Dataset
from ast import literal_eval
from pyproj import Proj, transform 
from cartopy import crs, feature
import ESMF
from matplotlib import cm, gridspec, rcParams
FORTRAN_CONTIGUOUS = True

def extract_pixel_coordinates(ULx,Uly,LRx,LRy,shape):
    in_proj  = Proj('+proj=sinu +R=6371007.181 +nadgrids=@null +wktext') #------ Specify input projection
    out_proj = Proj(init='epsg:4326')
    x        = np.linspace(ULx, LRx, shape[0], endpoint=False) + abs((ULx-LRx)/(2*shape[0]))
    y        = np.linspace(ULy, LRy, shape[0], endpoint=False) - abs((ULy-LRy)/(2*shape[0]))
    xx, yy   = np.meshgrid(x,y)
    plon, plat = transform(in_proj, out_proj, xx, yy)
    mlon, mlat = plon[0], np.array([i[0] for i in plat])
    plon       = plon.flatten()
    plat       = plat.flatten()
    return plon, plat, mlon, mlat

#### Read reflectance file and extract variables and coordinates 

In [14]:
shape    = 600
fc       = Dataset('MCD43A4/121/MCD43A4.A2018121.h10v05.006.2018130033209.hdf', mode='r')
struct   = getattr(fc, 'StructMetadata.0')
struct1  = struct[struct.find('UpperLeftPointMtrs'): struct.find('LowerRightMtrs')][19:-3]
struct2  = struct[struct.find('LowerRightMtrs')    : struct.find('Projection')    ][15:-3]
ULx, ULy = literal_eval(struct1)
LRx, LRy = literal_eval(struct2)
fc_lon, fc_lat, fc_mlon, fc_mlat  = extract_pixel_coordinates(ULx,ULy,LRx,LRy,(2400,2400))
lon_curv, lat_curv                = np.meshgrid(fc_mlon, fc_mlat)

nrb1a    = fc.variables['Nadir_Reflectance_Band1'][:]
fv_ref   = fc.variables['Nadir_Reflectance_Band1']._FillValue
lat1     = np.linspace(fc_mlat[0], fc_mlat[-1], shape, endpoint=True)# + abs((fc_mlat[0]-fc_mlat[-1])/(2*200))
lon1     = np.linspace(fc_mlon[0], fc_mlon[-1], shape, endpoint=True)# - abs((fc_mlon[0]-fc_mlon[-1])/(2*200))

lon_tar, lat_tar = np.meshgrid(lon1, lat1)

#### Resample original reflectance data (2400x2400) to desired shape (i.e. 1200x1200, 800x800, etc.) using PyResample

In [15]:
targ_def = pyresample.geometry.SwathDefinition(lons=lon_tar   , lats=lat_tar)
orig_def = pyresample.geometry.SwathDefinition(lons=lon_curv, lats=lat_curv)
wf       = lambda r: 1/r**2
nrb_resampled = pyresample.kd_tree.resample_custom(orig_def, nrb1a, targ_def, radius_of_influence=500000, neighbours=8, weight_funcs=wf, fill_value=float(fv_ref))

/home/kaushal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


#### Take output (resampled data) from PyResample and use it as source data for ESMF Regridding. The output will be obtained at SIF Locations 

In [23]:
sourcegrid = ESMF.Grid(np.array((shape,shape)), staggerloc=ESMF.StaggerLoc.CENTER, coord_sys=ESMF.CoordSys.SPH_DEG)
source_lon = sourcegrid.get_coords(0)
source_lat = sourcegrid.get_coords(1)

if FORTRAN_CONTIGUOUS:
    source_lon[...] = lon_tar.T
    source_lat[...] = lat_tar.T
else:
    source_lon[...] = lon_tar
    source_lat[...] = lat_tar

sourcefield = ESMF.Field(sourcegrid, name='Reflectance_Resampled')

if FORTRAN_CONTIGUOUS:
    sourcefield.data[...] = nrb_resampled.T
else:
    sourcefield.data[...] = nrb_resampled

sif_lat  = np.array([30.05, 38.23, 30.87 , 30.01, 38.88])
sif_lon  = np.array([-81  , -77  , -90.54, -88.4, -79.56])
sif_loc  = ESMF.LocStream(len(sif_lat), coord_sys = ESMF.CoordSys.SPH_DEG, name = 'sif_locs')
sif_loc['ESMF:Lat'] = sif_lat
sif_loc['ESMF:Lon'] = sif_lon
sif_data = ESMF.Field(sif_loc, name = 'Reflectance_Interpolated_at_Sif_Locations')
regrid   = ESMF.Regrid(sourcefield, sif_data, regrid_method = ESMF.RegridMethod.NEAREST_STOD, unmapped_action = ESMF.UnmappedAction.IGNORE)#,dst_mask_values=np.array([float(fv_ref)]))

sif_data = regrid(sourcefield, sif_data)

In [24]:
sif_data.data

array([0.00992049, 0.13952688, 0.05168508, 0.0163983 , 0.13270207])